In [180]:
import torch
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from PIL import Image
import pandas as pd
from torchvision import transforms
from sklearn.model_selection import train_test_split
import preprocessing
import torch.optim as optim

In [235]:
def split_data(df):
    train_ratio = 0.7
    val_ratio = 0.2
    test_ratio = 0.1

    train_val, test = train_test_split(df, test_size=test_ratio, random_state=42)
    train, val = train_test_split(train_val, test_size=val_ratio/(train_ratio + val_ratio), random_state=42)

    return train, val, test

In [236]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, data_path, transform=None):
        self.dataframe = dataframe
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]  # Assuming the image column is 'Image_ID'
        img_path = f"{self.data_path}/{img_name}"
        image = Image.open(img_path)
        age = torch.tensor(self.dataframe.iloc[idx,1])
        gender = torch.tensor(self.dataframe.iloc[idx,2])
        # labels = self.dataframe.iloc[idx, 1:3].values.astype('float32')  # Assuming labels start from column 1

        if self.transform:
            image = self.transform(image)

        return image, [age, gender]

# Define transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

def preprocess_data(data_path, batch_size=32):
    df = pd.read_csv('../data/UTKFace_labels_for_trying.csv', dtype={'Age':'float32', 'Gender':'float32'})
    train_data, val_data, test_data = split_data(df)  # Assuming you have a function that splits your data

    train_dataset = CustomDataset(dataframe=train_data, data_path=data_path, transform=transform)
    val_dataset = CustomDataset(dataframe=val_data, data_path=data_path, transform=transform)
    test_dataset = CustomDataset(dataframe=test_data, data_path=data_path)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader, test_loader


In [237]:
data_path = data_path = "../data/for_trying"
train_loader, val_loader, test_loader = preprocess_data(data_path)

In [238]:
# Iterate through the train_loader to get a batch of data
for batch_idx, (data, targets) in enumerate(train_loader):
    example_data = data  # This will contain a batch of images
    example_targets = targets  # This will contain the corresponding labels/targets
    break

In [239]:
tf.print(example_data)

tensor([[[[ 0.5294,  0.5373,  0.5529,  ...,  0.2157,  0.1843,  0.1608],
          [ 0.5216,  0.5216,  0.5373,  ...,  0.2157,  0.1843,  0.1608],
          [ 0.5216,  0.5216,  0.5294,  ...,  0.2235,  0.1922,  0.1686],
          ...,
          [-1.0000, -1.0000, -1.0000,  ...,  0.3176,  0.3255,  0.3176],
          [-1.0000, -1.0000, -1.0000,  ...,  0.3098,  0.3176,  0.3333],
          [-1.0000, -1.0000, -1.0000,  ...,  0.2863,  0.3020,  0.3255]],

         [[ 0.3569,  0.3647,  0.3804,  ..., -0.0118, -0.0431, -0.0667],
          [ 0.3490,  0.3490,  0.3725,  ..., -0.0118, -0.0431, -0.0667],
          [ 0.3490,  0.3490,  0.3647,  ..., -0.0039, -0.0353, -0.0588],
          ...,
          [-1.0000, -1.0000, -1.0000,  ...,  0.4902,  0.4902,  0.4824],
          [-1.0000, -1.0000, -1.0000,  ...,  0.4824,  0.4902,  0.4980],
          [-1.0000, -1.0000, -1.0000,  ...,  0.4667,  0.4745,  0.4902]],

         [[-0.0118, -0.0039,  0.0118,  ..., -0.3255, -0.3569, -0.3804],
          [-0.0196, -0.0196,  

In [240]:
tf.print(example_targets)

[tensor([1., 1., 1., 1., 1., 1.]), tensor([0., 0., 0., 0., 0., 0.])]


In [285]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=2)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.dense_shared = nn.Linear(64 * 64 * 64, 128)  # Calculate the input size based on your input_shape

        # Output layers
        self.classification_output = nn.Linear(128, 1)
        self.regression_output = nn.Linear(128, 1)

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Identity()  # No activation for linear output

    def forward(self, x):
        # Forward pass through convolutional layers
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.max_pool(x)

        # Flatten and pass through fully connected layers
        x = self.flatten(x)
        x = self.relu(self.dense_shared(x))

        # Classification branch
        classification_out = self.sigmoid(self.classification_output(x)).squeeze()

        # Regression branch
        regression_out = self.linear(self.regression_output(x)).squeeze()

        return regression_out, classification_out

In [286]:
class MultiTaskLossWrapper(nn.Module):
    def __init__(self, task_num):
        super(MultiTaskLossWrapper, self).__init__()
        self.task_num = task_num
        self.log_vars = nn.Parameter(torch.zeros((task_num)))

    def forward(self, age_pred, gen_pred, age_true, gen_true):

        mse, binCrossEntropy = nn.MSELoss(), nn.BCELoss()
        
        loss0 = mse(age_pred, age_true)
        loss1 = binCrossEntropy(gen_pred, gen_true)

        precision0 = torch.exp(-self.log_vars[0])
        loss0 = precision0*loss0 + self.log_vars[0]

        precision1 = torch.exp(-self.log_vars[1])
        loss1 = precision1*loss1 + self.log_vars[1]

        return loss0+loss1

In [287]:
model = CustomModel()
loss_func = MultiTaskLossWrapper(2)

In [288]:
# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate as needed

# Training loop
epochs = 10  # Define the number of epochs for training
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for batch_idx, (data, targets) in enumerate(train_loader):
        optimizer.zero_grad()  # Zero the gradients to prevent accumulation
        age_pred, gen_pred = model(data)  # Forward pass
        age_true = targets[0]
        gen_true = targets[1]
        print("age:",age_pred)
        print("gen:",gen_pred)
        # print(targets[0])
        # print(age_pred)
        # age = targets[:, 0]  # Assuming age is the first element in targets
        # gender = targets[:, 1]  # Assuming gender is the second element in targets
        # print(age)
        loss = loss_func(age_pred, gen_pred, age_true, gen_true)
        total_loss += loss.item()
        # Backpropagation
        loss.backward()
        optimizer.step()

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}")

age: tensor([-0.0620, -0.0452, -0.0320, -0.0347, -0.0545, -0.0559],
       grad_fn=<SqueezeBackward0>)
gen: tensor([0.4871, 0.4845, 0.4847, 0.4841, 0.4848, 0.4876],
       grad_fn=<SqueezeBackward0>)
Epoch [1/10], Loss: 1.7617
age: tensor([40.7081, 39.6753, 42.4140, 28.2802, 38.7958, 40.5531],
       grad_fn=<SqueezeBackward0>)
gen: tensor([3.5511e-08, 4.5376e-08, 8.0363e-09, 6.8981e-06, 7.6165e-08, 2.9831e-08],
       grad_fn=<SqueezeBackward0>)
Epoch [2/10], Loss: 1420.8022
age: tensor([5.0632, 5.1823, 4.9224, 5.0210, 3.9518, 4.6302],
       grad_fn=<SqueezeBackward0>)
gen: tensor([0.0006, 0.0005, 0.0008, 0.0005, 0.0037, 0.0004],
       grad_fn=<SqueezeBackward0>)
Epoch [3/10], Loss: 14.5755
age: tensor([-0.7016, -1.0727, -0.9125, -0.9241, -0.9962, -0.9614],
       grad_fn=<SqueezeBackward0>)
gen: tensor([0.0271, 0.0064, 0.0109, 0.0100, 0.0074, 0.0080],
       grad_fn=<SqueezeBackward0>)
Epoch [4/10], Loss: 3.7423
age: tensor([-0.4064, -0.5240, -0.4750, -0.6029, -0.5082, -0.4776],
  